# **Data Cleaning - Unemployment.xlsx - UnemploymentMedianIncome.csv**

# **Summary of Data Cleaning**
* GFS, GLDAS and population datasets were merged into one
* The final dataset is collected from 2015 April 1 to December June 30 with the below dates missing due to data absence:</br>
{
'2015-04-10', '2015-04-11', '2015-04-12', '2015-04-13', '2015-04-14', '2015-04-15', '2015-04-16', '2015-04-17', '2015-04-18', '2015-04-19', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31', '2022-12-31'
}
* 'SWE_tavg', 'SnowDepth_tavg', 'Qsb_tavg', 'Qsm_tavg', 'EvapSnow_tavg' were dropped
* A new feature **'water_availability'** is created and use it as a target variable (For details, see below **Feature Engineering** section)
* The dataset was split into train and test datasets: train --> 2015 to 2021, test --> 2022 & 2023
* Save them in train_bhopal.csv and test_bhopal.csv respectively
* train_bhopal.csv: 2435 rows, 28 columns
* test_bhopal.csv: 545 rows, 28 columns 

**Data Source:**
###### * https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25
###### * https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V022_CLSM_G025_DA1D


# **Import Modules**

In [1]:
#### Import the libraries needed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import os
import glob

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore')
%matplotlib notebook
%matplotlib inline

# **Set Environment**

In [2]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [3]:
# Set up directory
working_directory = Path.cwd()
# src = working_directory.parents[1]
# data directory
raw_data_directory = working_directory / 'data' / 'raw'
processed_data_directory = working_directory / 'data' / 'processed'
final_data_directory = working_directory / 'data' / 'final'

In [4]:
# Set pd.options to add slide bars
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [5]:
# Set default title color
plt.style.use('fivethirtyeight')

# set default plt figure size
plt.rcParams["figure.figsize"] = [10, 5]
# suptitle
plt.rcParams["figure.titlesize"] = 22
plt.rcParams["figure.titleweight"] = "bold"
plt.rcParams['text.color'] = '#333333'
# title
plt.rcParams["axes.titlesize"] = 16
plt.rcParams["axes.titleweight"] = "bold"
plt.rcParams["axes.titlelocation"] = "left"
plt.rcParams['axes.titlecolor'] = '#333333'
# label
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['axes.labelcolor'] = '#333333'
# spines
plt.rcParams["axes.spines.bottom"] = True
plt.rcParams["axes.spines.left"] = True
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False
# tick
plt.rcParams['xtick.color'] = "#333333"
plt.rcParams['ytick.color'] = "#333333"
# line width
plt.rcParams['lines.linewidth'] = 1

# **Load Unemployment.xlsx - UnemploymentMedianIncome.csv**
* 3277 rows and 100 columns
* Aggregated by the country(1), state(52), and county level

* Remove ',' and convert from string to float data type
* Dataset includes Port Rico --> delete
* Create a variable 'is_county'
* Remove "County" and the state abbrebiation Ex. Autauga County, AL --> Autauga
* Change the column name from "Area_Name" to "county"
* 149 rows 

* Data Dictionary: UrbanInfluenceCodes2013.xls, Ruralurbancontinuumcodes2023

In [160]:
# Convert the 'date' columns to datetime data type
file = os.path.join(raw_data_directory, 'Unemployment.xlsx - UnemploymentMedianIncome.csv')
unemployment_df = pd.read_csv(file, skiprows=4)

In [127]:
unemployment_df.shape

(3277, 100)

In [128]:
unemployment_df.head()

,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
0,0,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,"143,786,537","136,977,996","6,808,541",4.7,"144,839,298","136,455,783","8,383,515",5.8,"145,660,094","136,944,522","8,715,572",6.0,"146,724,795","138,613,904","8,110,891",5.5,"148,597,241","141,000,912","7,596,329",5.1,"150,707,773","143,729,350","6,978,423",4.6,"152,191,050","145,156,133","7,034,917",4.6,"153,761,037","144,860,349","8,900,688",5.8,"153,825,454","139,594,699","14,230,755",9.3,"154,254,521","139,393,814","14,860,707",9.6,"154,520,678","140,688,861","13,831,817",9.0,"155,038,121","142,527,201","12,510,920",8.1,"155,362,278","143,905,037","11,457,241",7.4,"155,936,159","146,318,952","9,617,207",6.2,"156,840,649","148,554,918","8,285,731",5.3,"158,674,951","150,949,349","7,725,602",4.9,"160,744,592","153,744,181","7,000,411",4.4,"162,039,448","155,727,509","6,311,939",3.9,"163,815,888","157,805,898","6,009,990",3.7,"161,483,724","148,453,335","13,030,389",8.1,"162,229,903","153,544,980","8,684,923",5.4,"164,781,642","158,766,998","6,014,644",3.7,"69,717",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,"2,128,027","2,017,467","110,560",5.2,"2,112,621","1,987,633","124,988",5.9,"2,128,668","2,001,568","127,100",6.0,"2,138,306","2,018,783","119,523",5.6,"2,140,356","2,045,234","95,122",4.4,"2,170,007","2,083,207","86,800",4.0,"2,180,448","2,092,030","88,418",4.1,"2,176,854","2,049,579","127,275",5.8,"2,156,593","1,938,784","217,809",10.1,"2,197,028","1,968,824","228,204",10.4,"2,202,337","1,991,379","210,958",9.6,"2,178,508","2,000,848","177,660",8.2,"2,172,102","2,012,828","159,274",7.3,"2,164,715","2,018,705","146,010",6.7,"2,152,295","2,020,443","131,852",6.1,"2,155,729","2,029,157","126,572",5.9,"2,203,458","2,103,873","99,585",4.5,"2,240,109","2,152,270","87,839",3.9,"2,272,935","2,200,437","72,498",3.2,"2,269,672","2,124,409","145,263",6.4,"2,259,349","2,183,330","76,019",3.4,"2,286,028","2,226,670","59,358",2.6,"53,990",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1

In [129]:
unemployment_df.tail()

,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
3272,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,"19,805","17,607","2,198",11.1,"19,554","17,138","2,416",12.4,"20,219","17,614","2,605",12.9,"20,789","18,031","2,758",13.3,"20,607","18,218","2,389",11.6,"20,930","18,296","2,634",12.6,"21,654","19,071","2,583",11.9,"21,115","18,445","2,670",12.6,"20,372","17,693","2,679",13.2,"19,955","16,433","3,522",17.6,"16,228","12,856","3,372",20.8,"15,293","12,430","2,863",18.7,"14,796","12,435","2,361",16.0,"14,601","12,198","2,403",16.5,"14,157","11,869","2,288",16.2,"13,820","11,921","1,899",13.7,"13,733","11,829","1,904",13.9,"13,414","11,746","1,668",12.4,"13,159","11,787","1,372",10.4,"13,174","11,912","1,262",9.6,NaN,NaN,NaN,NaN,"14,172","12,748","1,424",10.0,"14,232","13,180","1,052",7.4,NaN,NaN
3273,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,"2,441","2,120",321,13.2,"2,323","1,999",324,13.9,"2,625","2,159",466,17.8,"2,664","2,234",430,16.1,"2,554","2,237",317,12.4,"3,062","2,725",337,11.0,"3,300","2,992",308,9.3,"3,317","2,955",362,10.9,"3,214","2,811",403,12.5,"3,124","2,425",699,22.4,"3,401","2,767",634,18.6,"3,174","2,671",503,15.8,"3,043","2,651",392,12.9,"3,080","2,589",491,15.9,"3,152","2,710",442,14.0,"3,194","2,832",362,11.3,"3,260","2,914",346,10.6,"3,027","2,589",438,14.5,"2,726","2,276",450,16.5,"2,551","2,368",183,7.2,NaN,NaN,NaN,NaN,"2,804","2,469",335,11.9,"2,742","2,554",188,6.9,NaN,NaN
3274,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,"7,636","6,648",988,12.9,"7,564","6,546","1,018",13.5,"8,100","6,820","1,280",15.8,"8,393","7,134","1,259",15.0,"8,622","7,522","1,100",12.8,"9,506","8,112","1,394",14.7,"9,681","8,417","1,264",13.1,"9,479","8,223","1,256",13.3,"9,324","8,036","1,288",13.8,"9,518","7,628","1,890",19.9,"8,841","6,892","1,949",22.0,"8,485","6,681","1,804",21.3,"8,308","6,584","1,724",20.8,"8,184","6,416","1,768",21.6,"7,920","6,196","1,724",21.8,"7,802","6,262","1,540",19.7,"7,815","6,235","1,580",20.2,"7,557","6,090","1,467",19.4,"7,335","6,174","1,161",15.8,"7,494","6,309","1,185",15.8,NaN,NaN,NaN

In [130]:
unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3277 entries, 0 to 3276
Data columns (total 100 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPS_Code                                  3277 non-null   int64  
 1   State                                      3277 non-null   object 
 2   Area_Name                                  3277 non-null   object 
 3   Rural_Urban_Continuum_Code_2013            3219 non-null   float64
 4   Urban_Influence_Code_2013                  3219 non-null   float64
 5   Metro_2013                                 3224 non-null   float64
 6   Civilian_labor_force_2000                  3270 non-null   object 
 7   Employed_2000                              3270 non-null   object 
 8   Unemployed_2000                            3270 non-null   object 
 9   Unemployment_rate_2000                     3270 non-null   float64
 10  Civilian_labor_force_20

In [159]:
# The number of rows that contains NaN at least one
unemployment_df.isna().any(axis=1).sum()

149

In [131]:
unemployment_df.isna().sum()

FIPS_Code                                     0
State                                         0
Area_Name                                     0
Rural_Urban_Continuum_Code_2013              58
Urban_Influence_Code_2013                    58
Metro_2013                                   53
Civilian_labor_force_2000                     7
Employed_2000                                 7
Unemployed_2000                               7
Unemployment_rate_2000                        7
Civilian_labor_force_2001                     7
Employed_2001                                 7
Unemployed_2001                               7
Unemployment_rate_2001                        7
Civilian_labor_force_2002                     7
Employed_2002                                 7
Unemployed_2002                               7
Unemployment_rate_2002                        7
Civilian_labor_force_2003                     7
Employed_2003                                 7
Unemployed_2003                         

In [132]:
unemployment_df.describe().T

,count,mean,std,min,25%,50%,75%,max
FIPS_Code,3277.0,31318.964297,16332.539506,0.0,19019.0,30017.0,46099.000,72153.0
Rural_Urban_Continuum_Code_2013,3219.0,4.938490,2.724553,1.0,2.0,6.0,7.000,9.0
Urban_Influence_Code_2013,3219.0,5.189811,3.506942,1.0,2.0,5.0,8.000,12.0
Metro_2013,3224.0,0.382754,0.486135,0.0,0.0,0.0,1.000,1.0
Unemployment_rate_2000,3270.0,4.540581,2.042105,1.3,3.2,4.1,5.300,17.3
Unemployment_rate_2001,3270.0,5.233119,2.228461,1.6,3.8,4.8,6.100,18.6
Unemployment_rate_2002,3270.0,5.981193,2.344970,1.6,4.5,5.6,6.900,19.9
Unemployment_rate_2003,3270.0,6.226391,2.329511,1.9,4.7,5.8,7.300,20.2
Unemployment_rate_2004,3270.0,5.879235,2.114957,1.6,4.5,5.5,6.700,20.2
Unemployment_rate_2005,3263.0,5.671713,2.224239,2.0,4.2,5.2,6.500,21.0


In [133]:
unemployment_df.describe(include='object').T

,count,unique,top,freq
State,3277,53,TX,255
Area_Name,3277,3276,District of Columbia,2
Civilian_labor_force_2000,3270,3136,"2,986",3
Employed_2000,3270,3128,"3,902",3
Unemployed_2000,3270,1749,57,10
Civilian_labor_force_2001,3270,3127,"16,954",3
Employed_2001,3270,3118,"2,447",3
Unemployed_2001,3270,1891,196,11
Civilian_labor_force_2002,3270,3141,"5,677",4
Employed_2002,3270,3123,"4,353",3


## **Data Cleanup and Preparation**

* Remove ',' and convert from string to float data type

In [134]:
# Convert the datatype from object to float
object_columns = unemployment_df.select_dtypes(include=['object']).columns
# Remove 'State' and 'Area_Name'
object_columns = object_columns[2:]

In [135]:
# Remove "," to convert string to number
for col in object_columns:
    unemployment_df[col] = unemployment_df[col].str.replace(",", "")

unemployment_df[object_columns] = unemployment_df[object_columns].astype(float)

* Delete row whose 'State' is US --> delete 1 row
* Delete rows whose 'State' is Port Rico --> delete 79 rows

In [136]:
# Remove the country level
unemployment_df = unemployment_df[1:]

# Remove Port Rico, 79 rows
unemployment_df = unemployment_df[unemployment_df['State'] != "PR"].reset_index(drop=True)

* Create a variable 'is_state' --> If 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013' are all null, it's a state
* Create 'county' by removing " County" and " Parish", and the state abbrebiation from "Area_name": Ex. Autauga County, AL --> Autauga
* "Area_Name" in Alaska contains many area names ('Census Area', 'Borough', Borough/municipality, 'city/Census Area', 'city/Borough') --> Keep as is
* Delete 'Area_Name'

In [137]:
# Remove the state from Area_Name
county = []
for i in range(len(unemployment_df)):
    state = unemployment_df.loc[i, 'State']
    area = unemployment_df.loc[i, 'Area_Name'].split(f", {state}")[0]
    county.append(area)

In [138]:
# Create state flag
cols = ['Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013']
is_state = unemployment_df[cols].isna().all(axis=1)
unemployment_df['is_state'] = is_state

# Extract only county
unemployment_df['county'] = county
unemployment_df['county'] = unemployment_df['county'].str.replace(" County", "")
unemployment_df['county'] = unemployment_df['county'].str.replace(" Parish", "")

# Drop 'Area_Name'
unemployment_df.drop(['Area_Name'], axis=1, inplace=True)

* Change the column order starts from 'State', 'county', and 'is_county'

In [139]:
# Chnage column order
cols2 = unemployment_df.columns
cols2 = cols2[2:-2] # from 'Rural_Urban_Continuum_Code_2013' to 'Med_HH_Income_Percent_of_State_Total_2021'
cols1 = ['FIPS_Code', 'State', 'county', 'is_state']
cols1.extend(cols2) 
unemployment_df = unemployment_df[cols1]

## **Missing Value Analysis**

* 51 state rows
* State rows all nulls in 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013'

In [140]:
# Check null values at the state level
state_unemployment_df = unemployment_df[unemployment_df['is_state']]
num_state_rows = len(state_unemployment_df)
cols_w_missing_values = state_unemployment_df.columns[state_unemployment_df.isna().sum() > 0].values
print(f"THe number of state rows: {num_state_rows}")
print(f"Columns have null: {cols_w_missing_values}")
print()
print("These columns have all null values:")
print(state_unemployment_df[cols_w_missing_values].isna().all())
print()
# Verify there is no missing values in other columns
cols_wo_missing_values = list(set(state_unemployment_df.columns).difference(set(cols_w_missing_values)))
num_missing_values = state_unemployment_df[cols_wo_missing_values].isna().any().sum()
print(f"Number of null in other columns: {num_missing_values}")

THe number of state rows: 51
Columns have null: ['Rural_Urban_Continuum_Code_2013' 'Urban_Influence_Code_2013'
 'Metro_2013']

These columns have all null values:
Rural_Urban_Continuum_Code_2013    True
Urban_Influence_Code_2013          True
Metro_2013                         True
dtype: bool

Number of null in other columns: 0


* 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013' --> -1.0

In [141]:
# Impute 'Rural_Urban_Continuum_Code_2013' and 'Urban_Influence_Code_2013'
cols_rural_urban_metro = ['Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 'Metro_2013']
idx = (unemployment_df['is_state']) & (unemployment_df.isna().sum(axis=1)>0)
unemployment_df.loc[idx, cols_rural_urban_metro] = -1.0

In [142]:
# Check null values at the State level
unemployment_df[unemployment_df['is_state']].isna().sum().sum()

0

#### **Alaska**
* These counties in AK have null in 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013'
* 'Chugach Census Area', 'Copper River Census Area', 'Prince of Wales-Outer Ketchikan Census Area', 'Skagway-Hoonah-Angoon Census Area', 'Wrangell-Petersburg Census Area'
* These areas have been split and combined with other areas. Thus, they didn't exist in 2013. That's the reason the values are missing
* 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013' --> -1.0
#### **Luisiana**
* Unemployment-related features will be imputed with 0
* These parishes in AL have null in 2005 and 2006 unemployment-related features. This may be caused by harricane Katrina hit these counties in 2005
* 'Jefferson', 'Orleans', 'Plaquemines', 'St. Bernard', 'St. Charles', 'St. John the Baptist', 'St. Tammany'
* 'Civilian_labor_force_2005', 'Civilian_labor_force_2006', 'Unemployment_rate_2005', 'Unemployment_rate_2006' will be imputed the data obtained from homefacts.com
* Data Source: https://www.homefacts.com/unemployment/Louisiana/

|Parish|labor force 2005|labor force 2006|unemployment rate 2005|unemployment rate 2006|
|---|---|---|---|---|
|Jefferson|433336|436528|3.35|1.94|
|Orleans|409287|397096|4.23|2.81|
|Plaquemines|23481|23662|3.37|2.04|
|St. Bernard|51230|46948|3.77|2.41|
|St. Charles|53019|52798|3.56|2.0|
|St. John the Baptist|19486|16480|4.33|2.86|
|St. Tammany|96332|84484|2.94|1.70|

* https://www.homefacts.com/unemployment/Louisiana/
* LOUISIANA LABOR FORCE DIVERSITY DATA 2006
* Data Srouce: https://www.laworks.net/Downloads/Employment/AffirmativeActionPublication_2006.pdf
* Data Srouce: https://www.laworks.net/downloads/employment/affirmativeactionpublication_2007.pdf
* LOUISIANA LABOR FORCE DIVERSITY DATA 2007

In [143]:
# Check null at the county level
unemployment_df[~(unemployment_df['is_state']) & (unemployment_df.isna().sum(axis=1)>0)]

,FIPS_Code,State,county,is_state,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
74,2063,AK,Chugach Census Area,False,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3256.0,2973.0,283.0,8.7,3179.0,3100.0,79.0,2.5,3387.0,3290.0,97.0,2.9,88029.0,112.2
75,2066,AK,Copper River Census Area,False,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1293.0,1154.0,139.0,10.8,1246.0,1205.0,41.0,3.3,1405.0,1275.0,130.0,9.3,61288.0,78.1
80,2105,AK,Hoonah-Angoon Census Area,False,9.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1067.0,912.0,155.0,14.5,1097.0,928.0,169.0,15.4,1106.0,945.0,161.0,14.6,1119.0,954.0,165.0,14.7,1134.0,965.0,169.0,14.9,1128.0,962.0,166.0,14.7,1200.0,1064.0,136.0,11.3,1137.0,1006.0,131.0,11.5,1150.0,1016.0,134.0,11.7,1193.0,1069.0,124.0,10.4,1058.0,909.0,149.0,14.1,1152.0,1045.0,107.0,9.3,1251.0,1181.0,70.0,5.6,56555.0,72.1
91,2195,AK,Petersburg Borough,False,9.0,11.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1844.0,1692.0,152.0,8.2,1849.0,1691.0,158.0,8.5,1870.0,1716.0,154.0,8.2,1650.0,1507.0,143.0,8.7,1577.0,1430.0,147.0,9.3,1490.0,1361.0,129.0,8.7,1447.0,1322.0,125.0,8.6,1414.0,1292.0,122.0,8.6,1387.0,1276.0,111.0,8.0,1412.0,1298.0,114.0,8.1,1433.0,1302.0,131.0,9.1,1607.0,1494.0,113.0,7.0,1446.0,1362.0,84.0,5.8,71878.0,91.6
92,2198,AK,Prince of 

In [144]:
cols_rural_urban = ['Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013']
cols_unemployment = unemployment_df.columns[7:] # columns after 'Metro_2013'

* 'Rural_Urban_Continuum_Code_2013', 'Urban_Influence_Code_2013' --> -1.0

In [145]:
# Impute 'Rural_Urban_Continuum_Code_2013' and 'Urban_Influence_Code_2013'
idx = (unemployment_df['State'] == 'AK') & ~(unemployment_df['is_state']) & (unemployment_df[cols_rural_urban].isna().sum(axis=1)>0)
unemployment_df.loc[idx, cols_rural_urban] = -1.0

* Unemployment-related features will be imputed with 0

In [146]:
# Impute 'Rural_Urban_Continuum_Code_2013' and 'Urban_Influence_Code_2013'
idx = (unemployment_df['State'] == 'AK') & ~(unemployment_df['is_state']) & (unemployment_df[cols_unemployment].isna().sum(axis=1)>0)
unemployment_df.loc[idx, cols_unemployment] = 0

* 'Civilian_labor_force_2005', 'Civilian_labor_force_2006', 'Unemployment_rate_2005', 'Unemployment_rate_2006' will be imputed

In [147]:
# Load the missing data collected from homefacts.com
file = os.path.join(raw_data_directory, 'LA_missing_data.csv')
la_missing_data =  pd.read_csv(file)

In [148]:
la_missing_data

,Parish,Civilian_labor_force_2005,Civilian_labor_force_2006,Unemployment_rate_2005,Unemployment_rate_2006
0,Jefferson,433336,436528,5.03,3.88
1,Orleans,409287,397096,6.35,5.62
2,Plaquemines,23481,23662,5.59,4.08
3,St. Bernard,51230,46948,5.65,4.82
4,St. Charles,53019,52798,5.34,4.00
5,St. John the Baptist,19486,16480,6.50,5.72
6,St. Tammany,96332,84484,4.41,3.40


In [149]:
# Create featires
la_missing_data['Unemployed_2005'] = round(la_missing_data['Civilian_labor_force_2005'] * la_missing_data['Unemployment_rate_2005']/100)
la_missing_data['Employed_2005'] = la_missing_data['Civilian_labor_force_2005'] - la_missing_data['Unemployed_2005']
la_missing_data['Unemployed_2006'] = round(la_missing_data['Civilian_labor_force_2006'] * la_missing_data['Unemployment_rate_2006']/100)
la_missing_data['Employed_2006'] = la_missing_data['Civilian_labor_force_2006'] - la_missing_data['Unemployed_2006']

In [150]:
parish = la_missing_data['Parish']
cols_unemployment_2005_2006 = la_missing_data.columns[1:]

In [151]:
for i in range(len(la_missing_data)):
    county = la_missing_data['Parish'][i]
    for col in la_missing_data.columns[1:]:
        val = la_missing_data.loc[i, col]
        
        idx = (unemployment_df['State'] == 'LA') & (unemployment_df['county'] == county)
        unemployment_df.loc[idx, col] = val

In [152]:
# Verify no more null values
unemployment_df[(unemployment_df['State'].isin(['AK', 'LA'])) & ~(unemployment_df['is_state'])].isna().sum().sum()

0

In [153]:
# Check the final dataset
unemployment_df.head()

,FIPS_Code,State,county,is_state,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
0,1000,AL,Alabama,True,-1.0,-1.0,-1.0,2147173.0,2047731.0,99442.0,4.6,2128027.0,2017467.0,110560.0,5.2,2112621.0,1987633.0,124988.0,5.9,2128668.0,2001568.0,127100.0,6.0,2138306.0,2018783.0,119523.0,5.6,2140356.0,2045234.0,95122.0,4.4,2170007.0,2083207.0,86800.0,4.0,2180448.0,2092030.0,88418.0,4.1,2176854.0,2049579.0,127275.0,5.8,2156593.0,1938784.0,217809.0,10.1,2197028.0,1968824.0,228204.0,10.4,2202337.0,1991379.0,210958.0,9.6,2178508.0,2000848.0,177660.0,8.2,2172102.0,2012828.0,159274.0,7.3,2164715.0,2018705.0,146010.0,6.7,2152295.0,2020443.0,131852.0,6.1,2155729.0,2029157.0,126572.0,5.9,2203458.0,2103873.0,99585.0,4.5,2240109.0,2152270.0,87839.0,3.9,2272935.0,2200437.0,72498.0,3.2,2269672.0,2124409.0,145263.0,6.4,2259349.0,2183330.0,76019.0,3.4,2286028.0,2226670.0,59358.0,2.6,53990.0,100.0
1,1001,AL,Autauga,False,2.0,2.0,1.0,21861.0,20971.0,890.0,4.1,22081.0,21166.0,915.0,4.1,22161.0,21096.0,1065.0,4.8,22695.0,21557.0,1138.0,5.0,23241.0,22146.0,1095.0,4.7,23887.0,22986.0,901.0,3.8,24425.0,23619.0,806.0,3.3,24434.0,23610.0,824.0,3.4,24687.0,23376.0,1311.0,5.3,24660.0,22464.0,2196.0,8.9,25749.0,23481.0,2268.0,8.8,25845.0,23688.0,2157.0,8.3,25762.0,23932.0,1830.0,7.1,25783.0,24155.0,1628.0,6.3,25639.0,24150.0,1489.0,5.8,25541.0,24206.0,1335.0,5.2,25710.0,24395.0,1315.0,5.1,26269.0,25224.0,1045.0,4.0,26471.0,25515.0,956.0,3.6,26696.0,25927.0,769.0,2.9,26425.0,25023.0,1402.0,5.3,26545.0,25809.0,736.0,2.8,26789.0,26181.0,608.0,2.3,66444.0,123.1
2,1003,AL,Baldwin,False,3.0,2.0,1.0,69979.0,67370.0,2609.0,3.7,69569.0,66545.0,3024.0,4.3,69379.0,65881.0,3498.0,5.0,72598.0,69010.0,3588.0,4.9,74843.0,71061.0,3782.0,5.1,76608.0,73581.0,3027.0,4.0,79806.0,77263.0,2543.0,3.2,82829.0,80213.0,2616.0,3.2,83205.0,79222.0,3983.0,4.8,82314.0,74950.0,7364.0,8.9,83551.0,75283.0,8268.0,9.9,85077.0,77459.0,7618.0,9.0,84507.0,77973.0,6534.0,7.7,85206.0,79466.0,5740.0,6.7,86546.0,81265.0,5281.0,6.1,87493.0,82619.0,4874.0,5.6,89778.0,84972.0,4806.0,5.4,93152.0,89286.0,3866.0,4.2,96235.0,92734.0,3501.0

## **Save the Dataframe as CSV**

In [154]:
file = os.path.join(processed_data_directory, 'cleaned_UnemploymentMedianIncome.csv')
unemployment_df.to_csv(file, index=False)

## **Verify cleaned_UnemploymentMedianIncome.csv**

In [155]:
file = os.path.join(processed_data_directory, 'cleaned_UnemploymentMedianIncome.csv')
unemployment_df = pd.read_csv(file)
unemployment_df.head()

,FIPS_Code,State,county,is_state,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
0,1000,AL,Alabama,True,-1.0,-1.0,-1.0,2147173.0,2047731.0,99442.0,4.6,2128027.0,2017467.0,110560.0,5.2,2112621.0,1987633.0,124988.0,5.9,2128668.0,2001568.0,127100.0,6.0,2138306.0,2018783.0,119523.0,5.6,2140356.0,2045234.0,95122.0,4.4,2170007.0,2083207.0,86800.0,4.0,2180448.0,2092030.0,88418.0,4.1,2176854.0,2049579.0,127275.0,5.8,2156593.0,1938784.0,217809.0,10.1,2197028.0,1968824.0,228204.0,10.4,2202337.0,1991379.0,210958.0,9.6,2178508.0,2000848.0,177660.0,8.2,2172102.0,2012828.0,159274.0,7.3,2164715.0,2018705.0,146010.0,6.7,2152295.0,2020443.0,131852.0,6.1,2155729.0,2029157.0,126572.0,5.9,2203458.0,2103873.0,99585.0,4.5,2240109.0,2152270.0,87839.0,3.9,2272935.0,2200437.0,72498.0,3.2,2269672.0,2124409.0,145263.0,6.4,2259349.0,2183330.0,76019.0,3.4,2286028.0,2226670.0,59358.0,2.6,53990.0,100.0
1,1001,AL,Autauga,False,2.0,2.0,1.0,21861.0,20971.0,890.0,4.1,22081.0,21166.0,915.0,4.1,22161.0,21096.0,1065.0,4.8,22695.0,21557.0,1138.0,5.0,23241.0,22146.0,1095.0,4.7,23887.0,22986.0,901.0,3.8,24425.0,23619.0,806.0,3.3,24434.0,23610.0,824.0,3.4,24687.0,23376.0,1311.0,5.3,24660.0,22464.0,2196.0,8.9,25749.0,23481.0,2268.0,8.8,25845.0,23688.0,2157.0,8.3,25762.0,23932.0,1830.0,7.1,25783.0,24155.0,1628.0,6.3,25639.0,24150.0,1489.0,5.8,25541.0,24206.0,1335.0,5.2,25710.0,24395.0,1315.0,5.1,26269.0,25224.0,1045.0,4.0,26471.0,25515.0,956.0,3.6,26696.0,25927.0,769.0,2.9,26425.0,25023.0,1402.0,5.3,26545.0,25809.0,736.0,2.8,26789.0,26181.0,608.0,2.3,66444.0,123.1
2,1003,AL,Baldwin,False,3.0,2.0,1.0,69979.0,67370.0,2609.0,3.7,69569.0,66545.0,3024.0,4.3,69379.0,65881.0,3498.0,5.0,72598.0,69010.0,3588.0,4.9,74843.0,71061.0,3782.0,5.1,76608.0,73581.0,3027.0,4.0,79806.0,77263.0,2543.0,3.2,82829.0,80213.0,2616.0,3.2,83205.0,79222.0,3983.0,4.8,82314.0,74950.0,7364.0,8.9,83551.0,75283.0,8268.0,9.9,85077.0,77459.0,7618.0,9.0,84507.0,77973.0,6534.0,7.7,85206.0,79466.0,5740.0,6.7,86546.0,81265.0,5281.0,6.1,87493.0,82619.0,4874.0,5.6,89778.0,84972.0,4806.0,5.4,93152.0,89286.0,3866.0,4.2,96235.0,92734.0,3501.0